In [1]:
import os, sys
import wandb
from tqdm.notebook import tqdm
import transformers
import torch
import torch.nn as nn
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import DataLoader, Dataset, Subset
from importlib import reload
from torch_scatter import scatter
from transformers import pipeline
from IPython.display import clear_output

sys.path.append('../')

from utils import preprocess as pp
from utils.evaluate import eval_funcs, normalize
from utils.collate import collate_fn 
from utils.ckpt import _reload_best_model
from utils.graph_llm import GraphLLM
from utils.llm import LLM
from utils.multiplex import Multiplex
from utils.textualize import *
from utils.bio_graphs import BiologicalDataset

# Load Datasets

In [2]:
# get dataset
# base_path = '../data/DREAM4_gold_standards/'

# c_node_id_data = BiologicalDataset(base_path + 'connections_node_id')
# c_node_label_data = BiologicalDataset(base_path + 'connections_node_label')
# sp_node_id_data = BiologicalDataset(base_path + 'shortest_path_node_id')
# sp_node_label_data = BiologicalDataset(base_path + 'shortest_path_node_label')
# get dataset
data_path = '../data/subgraphs/all'
dataset = BiologicalDataset(data_path)
idx_split = dataset.get_idx_split()

In [ ]:
# split datasets on idx
train_dataset = Subset(dataset, idx_split['train'])
val_dataset = Subset(dataset, idx_split['val'])
test_dataset = Subset(dataset, idx_split['test'])

# options
batch_size = 8

# make dataloaders
train_loader = DataLoader(train_dataset, 
                          batch_size=batch_size,
                          shuffle=True,
                          collate_fn=collate_fn)

val_loader = DataLoader(val_dataset, 
                          batch_size=batch_size,
                          shuffle=True,
                          collate_fn=collate_fn)

test_loader = DataLoader(test_dataset, 
                          batch_size=batch_size,
                          shuffle=True,
                          collate_fn=collate_fn)

# Load Models

In [4]:
vanilla_llm = LLM(max_text_len=512,
                  max_max_new_tokens=32,
                  max_memory=[80, 80],
                  llm_model_path='meta-llama/Meta-Llama-3-8B-Instruct',
                  llm_frozen='True',
                  revision="main") # need toadd args

Loading LLaMA...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Freezing LLaMA...
Finished loading LLaMA...


In [5]:
# base model
base_graph_llm = GraphLLM(max_text_len=512,
                     max_max_new_tokens=32,
                     max_memory=[80, 80],
                     llm_model_path='meta-llama/Meta-Llama-3-8B-Instruct',
                     llm_frozen='True',
                     revision="main") # args are defaulted in the class

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model is frozen


In [6]:
# path
path = '../checkpoints/graph_llm_fsdp/epoch_1_best.pth'

# load model
trained_graph_llm = _reload_best_model(base_graph_llm, path)

Loading checkpoint from ../checkpoints/graph_llm_fsdp/epoch_1_best.pth.


/mnt/DGX01/Personal/krusepi/codebase/projects/llms/bio-graph-rag/notebooks/../utils/ckpt.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path, m

[2025-02-14 15:48:18,302] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/pkr/miniconda3/envs/rag/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/pkr/miniconda3/envs/rag/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlvsym'
/home/pkr/miniconda3/envs/rag/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlopen'
/home/pkr/miniconda3/envs/rag/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlclose'
/home/pkr/miniconda3/envs/rag/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlerror'
/home/pkr/miniconda3/envs/rag/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlsym'
collect2: error: ld returned 1 exit status


# Evaluate Node Prediction Accuracy

In [7]:
# options
verbose = False
model = trained_graph_llm
loader = test_loader

# set to eval
model.model.generation_config.pad_token_id = model.tokenizer.pad_token_id
model.eval()

n_correct = 0
i = 0
# loop through dataloader
for batch in tqdm(loader):
    out = model.inference(batch)

    pred = out['pred']
    actual = out['label']

    # test accuracy
    for p, a in zip(pred, actual):
        p_ans, p_think = normalize(p)
        a = str(a)
        if verbose:
            print(p_think)
            print(p_ans)
            print(a)
            print()
        if a in p_ans:
            n_correct += 1
            if verbose:
                print("Correct!")
                print()
        else:
            if verbose:
                print("Incorrect :(")
                print()
        i += 1
    print(f"Accuracy: {n_correct/i:.2%} | {n_correct}/{i}", end='\r')
        
acc = n_correct / i
print(f"Accuracy: {acc:.2%} | {n_correct}/{i}")


  0%|          | 0/2257 [00:00<?, ?it/s]

KeyboardInterrupt: 